In [ ]:
import tensorflow as tf
from tensorflow.keras.datasets import cifar100
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import ReduceLROnPlateau
import os

In [ ]:
pwd

'/content'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#os.mkdir("/content/drive/MyDrive/EE992/ResNet50v2/")
path = "/content/drive/MyDrive/EE992/ResNet50v2/"


In [ ]:


#CIFAR-100 dataset
(x_train, y_train), (x_test, y_test) = cifar100.load_data('fine')

x_train, x_val, y_train, y_val = train_test_split(
    x_train, y_train, test_size=0.2, random_state=42)

np.savez_compressed(path+"test_dataset.npz", images=x_test, labels=y_test)
np.savez_compressed(path+"train_dataset.npz", images=x_train, labels=y_train)
np.savez_compressed(path+"val_dataset.npz", images=x_val, labels=y_val)

In [ ]:
#convert pixel values to float32 and normalize to [0, 1]
x_train = x_train.astype('float32') / 255.0
x_val = x_val.astype('float32') / 255.0

y_train = tf.keras.utils.to_categorical(y_train, num_classes=100)
y_val = tf.keras.utils.to_categorical(y_val, num_classes=100)


#batch size
BATCH_SIZE = 256

#resizing images
def resize_images(image, label):
    image = tf.image.resize(image, [224, 224])
    return image, label

#augmenting images
def augment(image, label):
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_brightness(image, max_delta=0.2)
    image = tf.image.random_contrast(image, lower=0.8, upper=1.2)
    return image, label

In [ ]:

#training dataset pipeline
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_dataset = train_dataset.map(resize_images, num_parallel_calls=tf.data.AUTOTUNE)
train_dataset = train_dataset.map(augment, num_parallel_calls=tf.data.AUTOTUNE)
train_dataset = train_dataset.shuffle(buffer_size=10000).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

# Define the number of augmentations per image
# N = 2  # This will triple the dataset (original + 2 augmented)

# Expand dataset by repeating each image N times before augmentation
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
# train_dataset = train_dataset.flat_map(lambda img, label: tf.data.Dataset.from_tensors((img, label)).repeat(N))

# Apply transformations: Resize → Augment → Shuffle → Batch → Prefetch
train_dataset = train_dataset.map(resize_images, num_parallel_calls=tf.data.AUTOTUNE)
train_dataset = train_dataset.map(augment, num_parallel_calls=tf.data.AUTOTUNE)
train_dataset = train_dataset.shuffle(buffer_size=10000).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

#test dataset pipeline (only resizing, no augmentation)
test_dataset = tf.data.Dataset.from_tensor_slices((x_val, y_val))
test_dataset = test_dataset.map(resize_images, num_parallel_calls=tf.data.AUTOTUNE)
test_dataset = test_dataset.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)


In [ ]:
# #metrics
# from tensorflow.keras.metrics import Precision, Recall, Accuracy, SparseCategoricalAccuracy

# METRICS = [SparseCategoricalAccuracy(), Precision(), Recall()]

from tensorflow.keras.metrics import CategoricalAccuracy, Precision, Recall, AUC, F1Score

METRICS = [
    CategoricalAccuracy(name='accuracy'),
    Precision(name='precision'),
    Recall(name='recall'),
    AUC(name='auc'),
    F1Score(name='f1_score')
]

In [ ]:
def train_tl(base_model, train_dataset, val_dataset, OPT, METRICS ,DROPOUT = 0.5, LR = 1e-3, REG = 0.01, EPOCHS=30, opt_name='none'):

    for layer in base_model.layers:
        if hasattr(layer, "kernerl_regulaizer"):
            layer.kernel_regularizer = l2(REG)

    #freeze the base model
    base_model.trainable = False

    #add layers
    model = models.Sequential([
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.Dense(1024, activation='relu'),
        layers.Dropout(DROPOUT),
        layers.Dense(256, activation='relu'),
        layers.Dropout(DROPOUT),
        layers.Dense(100, activation='softmax')  # 100 classes
    ])

    #for plateau

    MIN_LR = LR/100

    #callbacks
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
    # checkpoint = tf.keras.callbacks.ModelCheckpoint(path+"best_model.keras", save_best_only=True, monitor='val_loss')
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=MIN_LR)

    model.compile(optimizer=OPT,
                  loss='categorical_crossentropy',
                  metrics=METRICS)

    #model.summary()

    model_name = base_model.name +'_' + opt_name + "_" + str(DROPOUT) + "_" + '_' + str(REG) + '_' + str(LR) + "_"

    checkpoint = tf.keras.callbacks.ModelCheckpoint(path+model_name+"tl.keras", save_best_only=True, monitor='val_loss')

    history = model.fit(train_dataset, epochs=EPOCHS, validation_data=(val_dataset),callbacks=[early_stopping, checkpoint, reduce_lr])

    # convert history to DataFrame
    history_df = pd.DataFrame(history.history)

    history_df.to_csv(path+model_name+"tl.csv", index=False)

     #load the saved best model
    fine_tune_model = tf.keras.models.load_model(path+model_name+"tl.keras")
    checkpoint = tf.keras.callbacks.ModelCheckpoint(path+model_name+"ft.keras", save_best_only=True, monitor='val_loss')
    #csv_logger = tf.keras.callbacks.CSVLogger(model_name+"ft.csv")


    #unfreeze some layers for fine-tuning
    for layer in fine_tune_model.layers[:-1]:  # Keep last layer frozen
        layer.trainable = True

    fine_tune_model.compile(optimizer=tf.keras.optimizers.AdamW(learning_rate=LR/50.0),
                            loss='categorical_crossentropy',
                            metrics=METRICS)

    #fine_tune_model.summary()

    fine_tune_history = fine_tune_model.fit(train_dataset, epochs=EPOCHS, validation_data=(val_dataset),callbacks=[early_stopping, checkpoint, reduce_lr])

     # Convert history to DataFrame
    history_df = pd.DataFrame(fine_tune_history.history)

     # Save to CSV
    history_df.to_csv(path+model_name+"ft.csv", index=False)




In [ ]:
#to remove warnings
# os.environ["XLA_FLAGS"] = "--xla_gpu_ptxas_flags=--warn-on-spills=false"
# os.environ["XLA_FLAGS"] += " --xla_gpu_ptxas_flags=--maxrregcount=64"

base_model = tf.keras.applications.ResNet50V2(input_shape=(224, 224, 3),
                                            include_top=False,
                                            weights='imagenet')

dropout = [0, 0.2, 0.6]
reg = [0, 0.01, 0.2]
lr = [1e-3, 1e-4, 1e-5]

for DROPOUT, REG in zip(dropout, reg):
    for LR in lr:
        decay = 0 if DROPOUT == 0 else 0.01
        OPT = tf.keras.optimizers.Adam(learning_rate=LR, weight_decay = decay)
        train_tl(base_model, train_dataset=train_dataset, val_dataset=test_dataset, METRICS = METRICS, OPT=OPT, DROPOUT=DROPOUT,
        LR=LR, REG = REG, EPOCHS=30, opt_name = 'adam')

        print('\n\n\n\n\n')

Epoch 1/30
157/157 ━━━━━━━━━━━━━━━━━━━━ 46s 204ms/step - accuracy: 0.2880 - auc: 0.8551 - f1_score: 0.2977 - loss: 2.4597 - precision: 0.7544 - recall: 0.1726 - val_accuracy: 0.5780 - val_auc: 0.9698 - val_f1_score: 0.5751 - val_loss: 1.5078 - val_precision: 0.7621 - val_recall: 0.4555 - learning_rate: 0.0010
Epoch 2/30
157/157 ━━━━━━━━━━━━━━━━━━━━ 20s 120ms/step - accuracy: 0.6383 - auc: 0.9799 - f1_score: 0.6349 - loss: 1.2587 - precision: 0.8156 - recall: 0.5149 - val_accuracy: 0.6153 - val_auc: 0.9725 - val_f1_score: 0.6102 - val_loss: 1.3559 - val_precision: 0.7811 - val_recall: 0.5177 - learning_rate: 0.0010
Epoch 3/30
157/157 ━━━━━━━━━━━━━━━━━━━━ 20s 120ms/step - accuracy: 0.6984 - auc: 0.9863 - f1_score: 0.6945 - loss: 1.0233 - precision: 0.8376 - recall: 0.5871 - val_accuracy: 0.6353 - val_auc: 0.9738 - val_f1_score: 0.6342 - val_loss: 1.2839 - val_precision: 0.7825 - val_recall: 0.5468 - learning_rate: 0.0010
Epoch 4/30
157/157 ━━━━━━━━━━━━━━━━━━━━ 19s 113ms/step - accuracy: 

In [ ]:
pwd


In [ ]:
    # #load the saved best model
    # fine_tune_model = model#tf.keras.models.load_model(model_name+"tl.keras")
    # checkpoint = tf.keras.callbacks.ModelCheckpoint(model_name+"ft.keras", save_best_only=True, monitor='val_loss')
    # #csv_logger = tf.keras.callbacks.CSVLogger(model_name+"ft.csv")


    # #unfreeze some layers for fine-tuning
    # for layer in fine_tune_model.layers[:-1]:  # Keep last layer frozen
    #     layer.trainable = True

    # fine_tune_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=LR/50.0),
    #                         loss='categorical_crossentropy',
    #                         metrics=METRICS)

    # #fine_tune_model.summary()

    # fine_tune_history = fine_tune_model.fit(train_dataset, epochs=EPOCHS, validation_data=(val_dataset),callbacks=[early_stopping, checkpoint, reduce_lr])

    #  # Convert history to DataFrame
    # history_df = pd.DataFrame(fine_tune_history.history)

    #  # Save to CSV
    # history_df.to_csv(model_name+"ft.csv", index=False)

